In [ ]:
import sys
!pip install datasets
!pip install spacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
import os
os.chdir('/content/drive/MyDrive/internal/mpng_cust/units/')
import utlts
from utlts import *

2.1.0+cu121


In [ ]:
%%time
data = pd.read_csv('/content/test_units.csv')
data['ln'] = data['source_code'].apply(lambda x: len(list(x.encode('ascii',errors='ignore'))))
data = data[data['ln'] < 60]
data['source_code'] = data['source_code'].apply(lambda x: clean_input(x))
dat = DatasetDict({
       'test':Dataset.from_dict({'label': data.concept_name,'text':data.source_code})
     })
tgts = []
preds = []
max_len = 60
max_len = 60

for datum in tqdm(dat['test']):
    src = datum["text"]
    tgt = datum["label"]
    src_idx = torch.tensor(text_transform_sou(src)).unsqueeze(0).to(device)
    pred_tgt = translate_seq(transformer_model, src_idx, device, max_len)
    pred_tgt = pred_tgt[1:-1]
    pred_sent = ' '.join([tar_vocab.get_itos()[i] for i in pred_tgt])
    preds.append(pred_sent)
    tgts.append(tgt.lower().split())


100%|██████████| 1344/1344 [00:52<00:00, 25.48it/s]

CPU times: user 51 s, sys: 263 ms, total: 51.2 s
Wall time: 52.8 s


In [ ]:
correct = 0
for i, j in zip(preds, data.concept_name.tolist()):
  if i == j.lower(): correct +=1

In [ ]:
correct / len(preds)

0.7857142857142857

In [ ]:
for n in range(20):
    print(f"Example {n+1}\n")
    ex = dat['test'][random.randint(0, len(dat['test']))]
    sentence = ex['text']
    src_indexes = torch.tensor(text_transform_sou(sentence)).unsqueeze(0).to(device)
    k = 3
    translated_sentence_ids = translate_seq_beam_search(transformer_model, src_indexes, k=k, device=device, max_len=50)
    translated_sentence_ids = sorted(translated_sentence_ids, key= lambda x: x[1], reverse=True)
    translations = [[tar_vocab.get_itos()[i] for i in translated_sentence[0]] for translated_sentence in translated_sentence_ids]
    print(f"Source : {ex['text']}")
    print(f"UCUM : {ex['label']}\n")
    print(f"Mapping generated:\n")
    for i in range(k):
        for w in translations[i]:
            if w in ['<sos>', '<eos>', '<pad>', '<unk>']:
                continue
            print(w, end=" ")
        print()
    print("---------------------------------------------------------------------\n")
del src_indexes, ex, sentence, translated_sentence_ids, translations
torch.cuda.empty_cache()
_ = gc.collect()

Example 1

Source : Gram.Deciliter
UCUM : gram per deciliter

Mapping generated:

gram per deciliter 
milligram per deciliter 
kilogram per deciliter 
---------------------------------------------------------------------

Example 2

Source : m2.g.s-3
UCUM : square meter gram per cubical second (10*-3 watt)

Mapping generated:

cubical meter gram per square second ( 10 * 3 cubical meter ) 
gram per square meter and cubical second ( 10 * -3 kilogram per second ) 
gram per square meter and cubical second ( 10 * -3 pascal per second ) 
---------------------------------------------------------------------

Example 3

Source : {MCG/ML}
UCUM : microgram per milliliter

Mapping generated:

microgram per milliliter 
milligram per liter 
microgram equivalent fibrinogen equivalent unit per milliliter 
---------------------------------------------------------------------

Example 4

Source : L/(24.h)
UCUM : liter per 24 hours

Mapping generated:

liter per 24 hours 
per 24 hours 
milliliter per 24